## Data gathering for project to predict the number of points an NBA player will socre on a given night

In [1]:
import nba_api.stats.endpoints as nba
import mysql.connector
from mysql.connector import errorcode
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



### Connect to database already built from previous project

In [2]:
dbname = 'nba'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname
)
cursor = cnx.cursor(buffered = True)

### Create tables for scoring, usage, and defensive boxscore stats as well as shot charts

In [114]:
tables = {}
tables['scoring'] = (
    "CREATE TABLE scoring ("
    " GAME_ID varchar(100),"
    " TEAM_ID int,"
    " TEAM_ABBREVIATION varchar(100),"
    " TEAM_CITY varchar(100),"
    " PLAYER_ID int,"
    " PLAYER_NAME varchar(100),"
    " START_POSITION varchar(2),"
    " COMMENT varchar(100),"
    " MIN varchar(100),"
    " PCT_FGA_2PT float,"
    " PCT_FGA_3PT float,"
    " PCT_PTS_2PT float,"
    " PCT_PTS_2PT_MR float,"
    " PCT_PTS_3PT float,"
    " PCT_PTS_FB float,"
    " PCT_PTS_FT float,"
    " PCT_PTS_OFF_TOV float,"
    " PCT_PTS_PAINT float,"
    " PCT_AST_2PM float,"
    " PCT_UAST_2PM float,"
    " PCT_AST_3PM float,"
    " PCT_UAST_3PM float,"
    " PCT_AST_FGM float,"
    " PCT_UAST_FGM float"
    ") ENGINE=InnoDB")

In [115]:
tables['box_usage'] = (
    "CREATE TABLE box_usage ("
    " GAME_ID varchar(100),"
    " TEAM_ID int,"
    " TEAM_ABBREVIATION varchar(100),"
    " TEAM_CITY varchar(100),"
    " PLAYER_ID int,"
    " PLAYER_NAME varchar(100),"
    " START_POSITION varchar(100),"
    " COMMENT varchar(100),"
    " MIN varchar(100),"
    " USG_PCT float,"
    " PCT_FGM float,"
    " PCT_FGA float,"
    " PCT_FG3M float,"
    " PCT_FG3A float,"
    " PCT_FTM float,"
    " PCT_FTA float,"
    " PCT_OREB float,"
    " PCT_DREB float,"
    " PCT_REB float,"
    " PCT_AST float,"
    " PCT_TOV float,"
    " PCT_STL float,"
    " PCT_BLK float,"
    " PCT_BLKA float,"
    " PCT_PF float,"
    " PCT_PFD float,"
    " PCT_PTS float"
    ") ENGINE=InnoDB")

In [116]:
tables['shot_chart'] = (
    " CREATE TABLE shot_chart ("
    " GRID_TYPE varchar(100),"
    " GAME_ID varchar(100),"
    " GAME_EVENT_ID int,"
    " PLAYER_ID int,"
    " PLAYER_NAME varchar(100),"
    " TEAM_ID int,"
    " TEAM_NAME varchar(100),"
    " PERIOD int,"
    " MINUTES_REMAINING int,"
    " SECONDS_REMAINING int,"
    " EVENT_TYPE varchar(100),"
    " ACTION_TYPE varchar(100),"
    " SHOT_TYPE varchar(100),"
    " SHOT_ZONE_BASIC varchar(100),"
    " SHOT_ZONE_AREA varchar(100),"
    " SHOT_ZONE_RANGE varchar(100),"
    " SHOT_DISTANCE int,"
    " LOC_X int,"
    " LOC_Y int,"
    " SHOT_ATTEMPTED_FLAG int,"
    " SHOT_MADE_FLAG int,"
    " GAME_DATE varchar(100),"
    " HTM varchar(100),"
    " VTM varchar(100)"
    ") ENGINE=InnoDB")


In [117]:
tables['defensive_boxscore'] = (
    "CREATE TABLE defensive_boxscore ("
    "GAME_ID varchar(100),"
    "TEAM_ID int,"
    "TEAM_ABBREVIATION varchar(100),"
    "TEAM_CITY varchar(100),"
    "TEAM_NICKNAME varchar(100),"
    "PLAYER_ID int,"
    "PLAYER_NAME varchar(100),"
    "START_POSITION varchar(100),"
    "COMMENT varchar(100),"
    "MATCHUP_MIN varchar(100),"
    "PARTIAL_POSS float,"
    "SWITCHES_ON int,"
    "PLAYER_PTS int,"
    "DREB int,"
    "MATCHUP_AST int,"
    "MATCHUP_TOV int,"
    "STL int,"
    "BLK int,"
    "MATCHUP_FGM int,"
    "MATCHUP_FGA int,"
    "MATCHUP_FG_PCT float,"
    "MATCHUP_FG3M int,"
    "MATCHUP_FG3A int,"
    "MATCHUP_FG3_PCT float"
    ") ENGINE=InnoDB")

In [118]:
for table_name in tables:
    table_description = tables[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table scoring: already exists.
Creating table box_usage: already exists.
Creating table shot_chart: already exists.
Creating table defensive_boxscore: already exists.


### Select all game ids from an already existing database from a previous project to retrieve advanced box score information

In [3]:
stmt = """SELECT GAME_ID FROM nba.game_logs"""
cursor.execute(stmt)
game_ids = cursor.fetchall()

In [4]:
game_ids = set(game_ids)
game_ids_df = pd.DataFrame(game_ids)

In [5]:
game_ids_df.columns = ['id']

### Build function to pull advanced box score information from the NBA API

In [124]:
def get_box_scoring(game_id):
    return nba.boxscorescoringv2.BoxScoreScoringV2(game_id=game_id).get_data_frames()[0]

### Call function on list of games from database

In [138]:
box_scores_scoring = []
count = 0
for game in game_ids_df['id'].values:
    box_scores_scoring.append(get_box_scoring(game))
    count+=1

KeyboardInterrupt: 

In [140]:
len(box_scores_scoring)

2163

In [144]:
box_scores_scoring[-1]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,PCT_FGA_2PT,...,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM
0,0021800597,1610612752,NYK,New York,203943,Noah Vonleh,F,,28:39,0.615,...,0.250,0.188,0.625,0.250,1.000,0.000,1.0,0.0,1.000,0.000
1,0021800597,1610612752,NYK,New York,1628995,Kevin Knox II,F,,21:35,0.667,...,0.250,0.625,0.250,0.000,0.000,0.000,1.0,0.0,1.000,0.000
2,0021800597,1610612752,NYK,New York,1628436,Luke Kornet,C,,18:50,0.500,...,0.000,0.333,0.000,0.667,0.000,1.000,0.0,0.0,0.000,1.000
3,0021800597,1610612752,NYK,New York,203501,Tim Hardaway Jr.,G,,26:44,0.417,...,0.000,0.000,0.000,0.000,1.000,0.000,0.0,1.0,0.500,0.500
4,0021800597,1610612752,NYK,New York,1626144,Emmanuel Mudiay,G,,34:41,0.941,...,0.647,0.176,0.000,0.824,0.000,1.000,0.0,0.0,0.000,1.000
5,0021800597,1610612752,NYK,New York,1626209,Mario Hezonja,,,31:11,0.800,...,0.143,0.286,0.143,0.429,0.600,0.400,0.0,0.0,0.600,0.400
6,0021800597,1610612752,NYK,New York,1628422,Damyean Dotson,,,21:38,0.500,...,0.000,0.000,0.000,0.000,1.000,0.000,1.0,0.0,1.000,0.000
7,0021800597,1610612752,NYK,New York,203504,Trey Burke,,,14:06,0.778,...,0.300,0.300,0.300,0.200,0.500,0.500,0.0,1.0,0.333,0.667
8,0021800597,1610612752,NYK,New York,202683,Enes Kanter,,,29:10,1.000,...,0.222,0.111,0.111,0.778,0.500,0.500,0.0,0.0,0.500,0.500
9,0021800597,1610612752,NYK,New York,1629019,Allonzo Trier,,,13:27,1.000,...,1.000,0.000,0.000,1.000,1.000,0.000,0.0,0.0,1.000,0.000


In [141]:
len(game_ids_df)

2699

In [145]:
game_ids_df['id'].values[2162]

'0021800597'

In [146]:
count=0
for game in game_ids_df['id'].values[2163:]:
    box_scores_scoring.append(get_box_scoring(game))
    count+=1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [147]:
len(box_scores_scoring)

2699

### Combine each dataframe returned from API call for insert  into database

In [148]:
# Concatenate box scores into dataframe, reset index
all_scoring = pd.concat(boxbox_scores_scoringcoring)
all_scoring.reset_index(drop = True, inplace = True)

### Build function that sets up a dataframe for insertion into database, builds an SQL insert statement based on the columns from the dataframe, and inserts the rows from the dataframe into the table

In [156]:
def insert_into_table(table, df):
    import numpy as np
    
    ### Get column names from table
    stmt = "SELECT * from nba.{}".format(table)
    cursor.execute(stmt)
    scores = cursor.fetchall()
    
    field_names = [i[0] for i in cursor.description]
    
    reordered_df = df[field_names]
    ## replace nan with none for SQL
    reordered_df.replace({np.nan: None}, inplace = True)
    
    ## build insert statement using properly ordered columns
    cols_insert = ''
    str_insert = ''
    for col in reordered_df.columns:
        cols_insert += col + ', '
        str_insert += '%s, '
    cols_insert = cols_insert.strip(', ')
    str_insert = str_insert.strip(', ')
    
    ## extract values from dataframe as tuples
    
    stats = [tuple(row) for row in reordered_df.values]
    
    # Insert and commit all the player stats
    insert_statement = """INSERT INTO nba.{} ({}) VALUES ({})""".format(table, cols_insert, str_insert)
    cursor.executemany(insert_statement, stats)
    cnx.commit()

In [ ]:
insert_into_table('scoring', all_scoring)

### Repeat above process for usage statistics

In [161]:
def get_usage(game_id):
    return nba.boxscoreusagev2.BoxScoreUsageV2(game_id=game_id).get_data_frames()[0]

In [166]:
usages = []
count = 0
for game in game_ids_df['id'].values:
    usages.append(get_usage(game))
    count+=1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [168]:
usages_all = pd.concat(usages)

In [170]:
usages_all.shape

(67817, 27)

In [171]:
insert_into_table('box_usage', usages_all)

### Repeat for defensive statistics

In [172]:
def get_defense(game_id):
    return nba.boxscoredefensive.BoxScoreDefensive(game_id=game_id).get_data_frames()[0]

In [175]:
defenses = []
count = 0
for game in game_ids_df['id'].values:
    defenses.append(get_defense(game))
    count+=1
    print (count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


SSLError: HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/boxscoredefensive?GameID=0021700671 (Caused by SSLError(SSLError("bad handshake: SysCallError(54, 'ECONNRESET')",),))

In [176]:
len(defenses)

523

In [177]:
count=0
for game in game_ids_df['id'][523:]:
    defenses.append(get_defense(game))
    count+=1
    print (count) 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [178]:
len(defenses)

2699

In [179]:
defense_all = pd.concat(defenses)

In [182]:
insert_into_table('defensive_boxscore', defense_all)

### The function to access shot chart data requires a team id and player id. Therefore I must select team ids and player ids for each game in the dataset to query the API

In [7]:
stmt = "SELECT TEAM_ID, PLAYER_ID FROM box_score"
cursor.execute(stmt)
team_player_combos = cursor.fetchall()

In [8]:
len(team_player_combos)

67818

### Most of the combinations of teams and players are duplicates (unless a player switches teams). I only need to keep the set of unique team and player combinations

In [9]:
unique = list(set(team_player_combos))

In [10]:
len(unique)

1172

### Function to retrieve shot chart data for a player

In [11]:
def get_shot_chart(team_id, player_id, date_from='', date_to=''):
    return nba.ShotChartDetail(team_id=team_id,player_id=player_id, date_from_nullable=date_from, date_to_nullable=date_to).get_data_frames()[0]


### Apply the function to my list of unique team/player combinations in my dataset

In [12]:
get_shot_chart(unique[0][0], unique[0][1], date_from ='10-17-2017', date_to = '11-24-2019')

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021700898,657,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,1,19,...,Center(C),Less Than 8 ft.,2,17,20,1,1,20180226,BOS,MEM
1,Shot Chart Detail,0021700898,671,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,0,12,...,Center(C),Less Than 8 ft.,2,-18,10,1,1,20180226,BOS,MEM
2,Shot Chart Detail,0021700931,133,1627744,Brice Johnson,1610612763,Memphis Grizzlies,1,1,39,...,Right Side(R),8-16 ft.,15,155,-1,1,1,20180302,MEM,DEN
3,Shot Chart Detail,0021700931,461,1627744,Brice Johnson,1610612763,Memphis Grizzlies,3,2,33,...,Center(C),Less Than 8 ft.,1,-8,8,1,1,20180302,MEM,DEN
4,Shot Chart Detail,0021700931,478,1627744,Brice Johnson,1610612763,Memphis Grizzlies,3,1,4,...,Center(C),Less Than 8 ft.,2,-12,17,1,1,20180302,MEM,DEN
5,Shot Chart Detail,0021700936,293,1627744,Brice Johnson,1610612763,Memphis Grizzlies,2,2,11,...,Center(C),Less Than 8 ft.,4,31,26,1,1,20180303,ORL,MEM
6,Shot Chart Detail,0021700989,598,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,4,10,...,Center(C),Less Than 8 ft.,3,-29,17,1,1,20180310,DAL,MEM
7,Shot Chart Detail,0021700989,619,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,1,55,...,Left Side(L),8-16 ft.,10,-78,73,1,1,20180310,DAL,MEM
8,Shot Chart Detail,0021701066,544,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,10,20,...,Center(C),Less Than 8 ft.,2,1,22,1,1,20180321,PHI,MEM
9,Shot Chart Detail,0021701066,562,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,9,17,...,Center(C),Less Than 8 ft.,3,-2,30,1,1,20180321,PHI,MEM


In [13]:
shot_charts = []
count = 0
for combo in unique:
    shot_charts.append(get_shot_chart(combo[0], combo[1], date_from ='10-17-2017', date_to = '11-24-2019' ))
    count+=1
    print (count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [14]:
len(shot_charts)

298

In [15]:
count = 0
for combo in unique[298:]:
    shot_charts.append(get_shot_chart(combo[0], combo[1], date_from ='10-17-2017', date_to = '11-24-2019' ))
    count+=1
    print (count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
len(shot_charts)

542

In [234]:
shot_charts[-1]

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021700013,32,1626161,Willie Cauley-Stein,1610612758,Sacramento Kings,1,9,19,...,Center(C),Less Than 8 ft.,0,7,6,1,1,20171018,SAC,HOU
1,Shot Chart Detail,0021700013,44,1626161,Willie Cauley-Stein,1610612758,Sacramento Kings,1,8,30,...,Center(C),Less Than 8 ft.,1,7,8,1,1,20171018,SAC,HOU
2,Shot Chart Detail,0021700013,51,1626161,Willie Cauley-Stein,1610612758,Sacramento Kings,1,7,34,...,Center(C),Less Than 8 ft.,2,5,20,1,1,20171018,SAC,HOU
3,Shot Chart Detail,0021700013,241,1626161,Willie Cauley-Stein,1610612758,Sacramento Kings,2,4,54,...,Center(C),8-16 ft.,9,35,88,1,1,20171018,SAC,HOU
4,Shot Chart Detail,0021700013,278,1626161,Willie Cauley-Stein,1610612758,Sacramento Kings,2,0,45,...,Center(C),Less Than 8 ft.,2,14,21,1,1,20171018,SAC,HOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,Shot Chart Detail,0021801207,215,1626161,Willie Cauley-Stein,1610612758,Sacramento Kings,2,8,11,...,Right Side(R),8-16 ft.,13,133,5,1,1,20190407,SAC,NOP
796,Shot Chart Detail,0021801207,246,1626161,Willie Cauley-Stein,1610612758,Sacramento Kings,2,5,40,...,Center(C),Less Than 8 ft.,1,-8,8,1,1,20190407,SAC,NOP
797,Shot Chart Detail,0021801230,8,1626161,Willie Cauley-Stein,1610612758,Sacramento Kings,1,11,23,...,Center(C),Less Than 8 ft.,1,3,14,1,1,20190410,POR,SAC
798,Shot Chart Detail,0021801230,61,1626161,Willie Cauley-Stein,1610612758,Sacramento Kings,1,6,59,...,Center(C),Less Than 8 ft.,0,6,6,1,1,20190410,POR,SAC


In [18]:
unique.pop[542]

(None, None)

In [17]:
count = 0
for combo in unique[542:]:
    shot_charts.append(get_shot_chart(combo[0], combo[1], date_from ='10-17-2017', date_to = '11-24-2019' ))
    count+=1
    print (count)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [237]:
len(shot_charts)

903

In [238]:
count = 0
for combo in unique[903:]:
    shot_charts.append(get_shot_chart(combo[0], combo[1], date_from ='10-17-2017', date_to = '11-24-2019' ))
    count+=1
    print (count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268


In [ ]:
len(shot_charts)